In [ ]:
# pip install arize-phoenix

In [1]:
import os
import sys

# --- 1. Observability Imports (Phoenix & OpenInference) ---
import phoenix as px
from phoenix.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.crewai import CrewAIInstrumentor

# --- 2. CrewAI Imports ---
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

# --- 3. Configuration & Tracing Setup ---
# The default Phoenix UI port is 6006.
TRACING_ENDPOINT = "http://localhost:6006/v1/traces"
PROJECT_NAME = "crewai-experiment-v2"

# Register the tracer provider with Phoenix
tracer_provider = register(
    project_name=PROJECT_NAME,
    endpoint=TRACING_ENDPOINT
)

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

# Instrument CrewAI: Captures high-level Agent/Task logic
CrewAIInstrumentor().instrument(
    tracer_provider=tracer_provider,
    skip_dep_check=True
)

# --- 4. Define the Experiment Logic ---

def run_crewai_experiment(prompt_variant: str):
    """
    Runs a single iteration of the crew with a specific prompt.
    """
    
    # ✅ FIXED: Fail fast if critical keys are missing
    if not os.getenv("SERPER_API_KEY") or not os.getenv("OPENAI_API_KEY"):
        print("⚠️ Error: Missing API Keys. Ensure SERPER_API_KEY and OPENAI_API_KEY are set.")
        sys.exit(1)
    
    # Initialize Tools
    search_tool = SerperDevTool()
    
    # Define LLM explicitly
    my_llm = LLM(
        model="gpt-4o-mini",
        temperature=0.2
    )
    
    # --- Define Agents ---
    
    researcher = Agent(
        role='Senior Research Analyst',
        goal='Uncover cutting-edge developments in AI and data science',
        backstory=(
            "You work at a leading tech think tank. "
            "Your expertise lies in identifying emerging trends. "
            "You have a knack for dissecting complex data and presenting actionable insights."
        ),
        verbose=True,
        allow_delegation=False,
        llm=my_llm,
        tools=[search_tool]
    )

    writer = Agent(
        role='Tech Content Strategist',
        goal='Craft compelling content on tech advancements',
        backstory=(
            "You are a renowned Content Strategist, known for your insightful and engaging articles. "
            "You transform complex concepts into compelling narratives."
        ),
        verbose=True,
        llm=my_llm,
        allow_delegation=False
    )
    
    # --- Define Tasks (With Critical Tracing Fixes) ---
    
    # ✅ FIXED: context=[] prevents OpenInference crash on the first task
    task1 = Task(
        description=prompt_variant,
        expected_output="Full analysis report in bullet points",
        agent=researcher,
        context=[] 
    )
    
    task2 = Task(
        description=(
            "Using the insights provided, develop an engaging blog post. "
            "Make it sound cool, avoid complex words so it doesn't sound like AI."
        ),
        expected_output="Full blog post of at least 4 paragraphs",
        agent=writer,
        context=[task1] # Passes data from researcher to writer
    )
    
    # --- Define & Run Crew ---
    
    crew = Crew(
        agents=[researcher, writer],
        tasks=[task1, task2],
        verbose=True,
        process=Process.sequential
    )
    
    print(f"\n🚀 Kickoff with prompt: {prompt_variant[:40]}...")
    result = crew.kickoff()
    
    return result

# --- 5. Main Execution Loop ---

if __name__ == "__main__":
    
    # Ensure Phoenix is running before starting
    print("ℹ️  Ensure your Phoenix server is running: 'Go to Terminal and Type phoenix serve or type python -m phoenix.server.main serve'\n")

    prompt_variants = [
        # 1. The Direct Command (Standard)
        "Conduct a comprehensive analysis of the latest advancements in AI in 2024.",

        # 2. The Executive Request (Focus on "Report" structure)
        "Generate a detailed report summarizing the most significant AI developments and breakthroughs of 2024.",

        # 3. The Synthesis Approach (Focus on connecting dots)
        "Synthesize a review of the state of AI innovation in 2024, highlighting major milestones and technical leaps.",

        # 4. The Investigative Approach (Focus on discovery)
        "Investigate the field of Artificial Intelligence in 2024 and document the key technological strides made this year.",

        # 5. The Landscape/Mapping Approach (Focus on breadth)
        "Map out the landscape of AI progress in 2024, identifying the top trends and new capabilities released to the public."
    ]

    results = []
    
    try:
        for i, variant in enumerate(prompt_variants):
            output = run_crewai_experiment(variant)
            results.append({"variant": variant, "output": output})
            
            # ✅ ADDED: Save output to file for easier reading
            filename = f"blog_output_{i+1}.md"
            with open(filename, "w") as f:
                f.write(str(output))
            
            print("\n------------------------------------------------")
            print(f"✅ Experiment Finished. Saved to {filename}")
            print("------------------------------------------------")
            
    except Exception as e:
        print(f"\n❌ Error during execution: {e}")

    # Final Summary
    print(f"\n🎉 Completed {len(results)} experiments.")
    print(f"👀 View traces at: http://localhost:6006")

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: crewai-experiment-v2
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

ℹ️  Ensure your Phoenix server is running: 'Go to Terminal and Type phoenix serve or type python -m phoenix.server.main serve'


🚀 Kickoff with prompt: Conduct a comprehensive analysis of the ...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ae0b97a0-6bf0-42ef-9353-10d245ca3413                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **The Future of AI: Navigating the Exciting Landscape of 2024**                                                │
│                                                                                                                 │
│  As we step into 2024, the world of artificial intelligence (AI) is buzzing with innovation and                 │
│  transformation. This year, the spotlight shines brightly on AI governance and ethics. With the rapid           │
│  deployment of AI technologies, companies are stepping up to ensure that these powerful tools are used          │
│  responsibly. Stricter guidelines are being put in place to tackle issues like bias and transparency, making    │
│  it clear that ethical AI practices are not just an afterthought but a fundamental part of the development      │
│  process. This shift is crucial as it helps build trust with users and ensures that AI serves everyone fairly.  │
│                                                                                                                 │
│  In the realm of creativity, generative AI is making waves like never before. Imagine a world where AI can      │
│  create stunning visuals, write engaging stories, or even compose music that resonates with our emotions. This  │
│  is not just a dream; it’s happening now! Industries such as marketing, entertainment, and design are adopting  │
│  generative AI tools to enhance their creative processes. The result? More realistic and captivating content    │
│  that captures the imagination. As these tools become more accessible, we can expect a surge in creativity      │
│  that blurs the lines between human and machine-generated art.                                                  │
│                                                                                                                 │
│  The impact of AI is also being felt in scientific research, where it’s accelerating discoveries at an          │
│  unprecedented pace. From drug discovery to climate modeling, AI is analyzing massive datasets to uncover       │
│  insights that were previously hidden. Researchers are leveraging AI to refine their methodologies, leading to  │
│  breakthroughs that could change the world. This fusion of technology and science is not just about speed;      │
│  it’s about enhancing our understanding of complex problems and finding innovative solutions to some of         │
│  humanity’s biggest challenges.                                                                                 │
│                                                                                                                 │
│  As we embrace these advancements, the rise of low-code and no-code platforms is democratizing AI development.  │
│  No longer do you need to be a coding wizard to create AI applications; these platforms empower anyone,         │
│  regardless of their technical background, to build and deploy AI solutions. This trend is opening doors for    │
│  organizations of all sizes to harness the power of AI, fostering a culture of innovation and creativity. With  │
│  AI tools becoming standard in workplaces, enhancing productivity and streamlining operations, the future       │
│  looks bright for both businesses and individuals ready to embrace this technological revolution.               │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a80259a6-93ce-402b-b4a2-bd9135d1a957                                                                     │
│  Agent: Tech Content Strategist                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6d83434c-03ec-4e11-bab5-f4b3aa4a2a2a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **The Future of AI: Navigating the Exciting Landscape of 2024**                                  │
│                                                                                                                 │
│  As we step into 2024, the world of artificial intelligence (AI) is buzzing with innovation and                 │
│  transformation. This year, the spotlight shines brightly on AI governance and ethics. With the rapid           │
│  deployment of AI technologies, companies are stepping up to ensure that these powerful tools are used          │
│  responsibly. Stricter guidelines are being put in place to tackle issues like bias and transparency, making    │
│  it clear that ethical AI practices are not just an afterthought but a fundamental part of the development      │
│  process. This shift is crucial as it helps build trust with users and ensures that AI serves everyone fairly.  │
│                                                                                                                 │
│  In the realm of creativity, generative AI is making waves like never before. Imagine a world where AI can      │
│  create stunning visuals, write engaging stories, or even compose music that resonates with our emotions. This  │
│  is not just a dream; it’s happening now! Industries such as marketing, entertainment, and design are adopting  │
│  generative AI tools to enhance their creative processes. The result? More realistic and captivating content    │
│  that captures the imagination. As these tools become more accessible, we can expect a surge in creativity      │
│  that blurs the lines between human and machine-generated art.                                                  │
│                                                                                                                 │
│  The impact of AI is also being felt in scientific research, where it’s accelerating discoveries at an          │
│  unprecedented pace. From drug discovery to climate modeling, AI is analyzing massive datasets to uncover       │
│  insights that were previously hidden. Researchers are leveraging AI to refine their methodologies, leading to  │
│  breakthroughs that could change the world. This fusion of technology and science is not just about speed;      │
│  it’s about enhancing our understanding of complex problems and finding innovative solutions to some of         │
│  humanity’s biggest challenges.                                                                                 │
│                                                                                                                 │
│  As we embrace these advancements, the rise of low-code and no-code platforms is democratizing AI development.  │
│  No longer do you need to be a coding wizard to create AI applications; these platforms empower anyone,         │
│  regardless of their technical background, to build and deploy AI solutions. This trend is opening doors for    │
│  organizations of all sizes to harness the power of AI, fostering a culture of innovation and creativity. With  │
│  AI tools becoming standard in workplaces, enhancing productivity and streamlining operations, the future       │
│  looks bright for both businesses and individuals read


------------------------------------------------
✅ Experiment Finished. Saved to blog_output_5.md
------------------------------------------------

🎉 Completed 5 experiments.
👀 View traces at: http://localhost:6006


In [2]:
CLARITY_LLM_JUDGE_PROMPT = """
In this task, you will be presented with a query and an answer. Your objective is to evaluate the clarity 
of the answer in addressing the query. A clear response is one that is precise, coherent, and directly 
addresses the query without introducing unnecessary complexity or ambiguity. An unclear response is one 
that is vague, disorganized, or difficult to understand, even if it may be factually correct.

Your response should be a single word: either "clear" or "unclear," and it should not include any other 
text or characters. "clear" indicates that the answer is well-structured, easy to understand, and 
appropriately addresses the query. "unclear" indicates that some part of the response could be better 
structured or worded.
Please carefully consider the query and answer before determining your response.

After analyzing the query and the answer, you must write a detailed explanation of your reasoning to 
justify why you chose either "clear" or "unclear." Avoid stating the final label at the beginning of your 
explanation. Your reasoning should include specific points about how the answer does or does not meet the 
criteria for clarity.

[BEGIN DATA]
Query: {variant}
Answer: {response}
[END DATA]
Please analyze the data carefully and provide an explanation followed by your response.

EXPLANATION: Provide your reasoning step by step, evaluating the clarity of the answer based on the query.
LABEL: "clear" or "unclear"
"""

In [3]:
from phoenix.trace import suppress_tracing
import pandas as pd
from phoenix.evals import llm_classify, OpenAIModel
import nest_asyncio
nest_asyncio.apply()
clarity_df = pd.DataFrame({
    "variant": [r["variant"] for r in results],
    "response": [r["output"] for r in results]
})
 
clarity_eval = llm_classify(
    data=clarity_df,    
    template=CLARITY_LLM_JUDGE_PROMPT,
    rails=['clear', 'unclear'],
    model=OpenAIModel(model="gpt-4o-mini"),
    provide_explanation=True
)




llm_classify |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

In [8]:
clarity_eval.to_csv('eval.csv')

In [11]:
clarity_eval['score'] = clarity_eval['label'].apply(lambda x: 1 if x == 'clear' else 0)
clarity_eval

,label,explanation,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens,score
0,unclear,The answer provided addresses the query about ...,[],COMPLETED,3.397341,913,133,1046,0
1,clear,The answer provides a comprehensive overview o...,[],COMPLETED,3.370009,1350,131,1481,1
2,clear,The answer provides a comprehensive overview o...,[],COMPLETED,3.368017,839,128,967,1
